In [1]:
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import polyline
from shapely import Point
import helper_functions as helper

In [2]:
basemap = gpd.read_file(r'../../data/SP1/MasterPlan2019PlanningAreaBoundaryNoSea.geojson')
basemap['Planning_Area'] = basemap["Description"].apply(lambda x: helper.extract_td_contents(x)[0])
basemap['geometry'] = basemap['geometry'].to_crs("4326")
mrt_stations_df = pd.read_csv(r"../../data/Cluster_data/mrt_station_final.csv",usecols = [1,2,3])
ranking = pd.read_csv(r'../../data/Cluster_data/mrt_ranking.csv')
mrt_stations_df_combined = pd.merge(mrt_stations_df, ranking, left_on='MRT.Name', right_on='MRT.Name', how='left')
for index, row in mrt_stations_df_combined.iterrows():
    point_coordinate = Point(row['Longitude'], row['Latitude'])
    Planning_Area = basemap[basemap.geometry.contains(point_coordinate)]['Planning_Area'].reset_index(drop=True)
    mrt_stations_df_combined.loc[index, 'Planning_Area'] = Planning_Area.values[0]

In [3]:
mrt_stations_df_combined.head()
# = mrt_stations_df_combined['Ranking'].fillna(0)
choices=["Distance","Suitability", "Time Savings", 'Time Savings(Log)',"Weighted Score"]
metrics = ["time_difference", "distance", "suitability", "station_density", "Weighted_Score"]

In [4]:
import plotly.graph_objects as go
import plotly.io as pio

# Optionally set a global theme
pio.templates.default = "simple_white"

def plot_planning_area_rankings(df, main_metric):
    # Ensure the main_metric is available.
    # Exclude MRT stations with no routes
    df = df.dropna(subset=[main_metric])
    
    # Group by the Planning Area and calculate mean for metrics
    df_grouped = df.groupby('Planning_Area', as_index=False).mean()
    df_grouped = df_grouped.sort_values(by=main_metric, ascending=False)
    
    # Initialize figure
    fig = go.Figure()
    fig.update_layout(title_text='Average ' + main_metric + ' for each Planning Area',
                      xaxis_title="Planning Area",
                      yaxis_title=main_metric,
                      hovermode="closest")

    # Default color for all bars
    colors = ['#7c8981'] * len(df_grouped)

    # Highlight the top and bottom 3 planning areas
    colors[0:3] = ['#cb2721'] * 3
    colors[-3:] = ['#0b569d'] * 3

    # Create hover text
    hover_texts = []
    for _, row in df_grouped.iterrows():
        hover_text = f"Planning Area: {row['Planning_Area']}<br>"
        metrics = [main_metric, "time_difference", "distance", "suitability", "Weighted_Score"]
        for m in metrics:
            hover_text += f"{m}: {row.get(m, 'N/A'):.2f}<br>"
        hover_texts.append(hover_text)

    # Add bar trace
    fig.add_trace(go.Bar(x=df_grouped['Planning_Area'], y=df_grouped[main_metric],
                         marker_color=colors,
                         hoverinfo="text",
                         text=hover_texts))

    # Show figure
    fig.show()

In [9]:
import plotly.graph_objects as go

def plot_planning_area_rankings(df, main_metric):
    # Exclude MRT stations with no routes
    df = df.dropna(subset=[main_metric])
    
    # Group by the Planning Area and calculate mean for metrics
    df_grouped = df.groupby('Planning_Area', as_index=False).mean()
    df_grouped = df_grouped.sort_values(by=main_metric, ascending=False)
    
    # Initialize figure
    fig = go.Figure()
    fig.update_layout(title_text='Average ' + main_metric + ' for each Planning Area',
                      yaxis_title=main_metric,
                      hovermode="closest")

    # Default color for all bars
    colors = ['#7c8981'] * len(df_grouped)

    # Highlight the top and bottom 3 planning areas
    colors[0:3] = ['red'] * 3
    colors[-3:] = ['blue'] * 3

    # Add bar trace
    fig.add_trace(go.Bar(x=df_grouped['Planning_Area'], y=df_grouped[main_metric],
                         marker_color=colors))

    # Create annotations for each of the highlighted planning areas
    annotations = []
    # Combining the top 3 and bottom 3 indices into one list for iteration
    top_bottom_indices = list(range(3)) + list(range(-3, 0))
    for i in top_bottom_indices:
        annotations.append(dict(
            x=df_grouped['Planning_Area'].iloc[i],
            y=df_grouped[main_metric].iloc[i],
            text=df_grouped['Planning_Area'].iloc[i],  # The label text
            showarrow=True,
            arrowhead=1,
            ax=0,  # No offset
            ay=-40,  # Position above the bar
            font=dict(color='red' if i < 3 else 'blue', size=12),
            arrowcolor='red' if i < 3 else 'blue'
        ))

    # Add annotations to the figure
    fig.update_layout(annotations=annotations)

    # Show figure
    fig.show()

In [7]:
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio

# Optionally set a global theme
pio.templates.default = "simple_white"

def plot_planning_area_rankings(df, main_metric):
    # Exclude MRT stations with no routes
    df = df.dropna(subset=[main_metric])
    
    # Group by the Planning Area and calculate mean for metrics
    df_grouped = df.groupby('Planning_Area', as_index=False).mean()
    df_grouped = df_grouped.sort_values(by=main_metric, ascending=False)
    
    # Initialize figure
    fig = go.Figure()
    fig.update_layout(title_text='Average ' + main_metric + ' for each Planning Area',
                      yaxis_title=main_metric,
                      hovermode="closest")  # Set the theme here

    # Default color for all bars
    colors = ['#7c8981'] * len(df_grouped)

    # Highlight the top and bottom 3 planning areas
    colors[0:3] = ['#cb2721'] * 3
    colors[-3:] = ['#0b569d'] * 3

    # Create hover text
    hover_texts = []
    for _, row in df_grouped.iterrows():
        hover_text = f"Planning Area: {row['Planning_Area']}<br>"
        metrics = [main_metric, "time_difference", "distance", "suitability", "Weighted_Score"]
        for m in metrics:
            hover_text += f"{m}: {row.get(m, 'N/A'):.2f}<br>"
        hover_texts.append(hover_text)

    # Add bar trace
    fig.add_trace(go.Bar(x=df_grouped['Planning_Area'], y=df_grouped[main_metric],
                         marker_color=colors,
                         hoverinfo="text",
                         text=hover_texts))

    # Select the labels for the top and bottom 3 planning areas
    top_3 = df_grouped['Planning_Area'].head(3).tolist()
    bottom_3 = df_grouped['Planning_Area'].tail(3).tolist()
    selected_labels = top_3 + bottom_3

    # Set custom x-axis tick positions and labels
    fig.update_layout(
        xaxis=dict(showticklabels=False),
        annotations=[]  # Assuming you are not using annotations here, otherwise add your annotations list
    )
    # Create annotations for each of the highlighted planning areas
    annotations = []
    # Combining the top 3 and bottom 3 indices into one list for iteration
    top_bottom_indices = list(range(3)) + list(range(-3, 0))
    for i in top_bottom_indices:
        annotations.append(dict(
            x=df_grouped['Planning_Area'].iloc[i],
            y=df_grouped[main_metric].iloc[i],
            text=df_grouped['Planning_Area'].iloc[i],  # The label text
            showarrow=True,
            arrowhead=1,
            ax=0,  # No offset
            ay=-40,  # Position above the bar
            font=dict(color='red' if i < 3 else 'blue', size=12),
            arrowcolor='red' if i < 3 else 'blue'
        ))

    # Add annotations to the figure
    fig.update_layout(annotations=annotations)



    # Show figure
    fig.show()

In [10]:
plot_planning_area_rankings(mrt_stations_df_combined, "suitability")

/var/folders/lk/jtg57k7555qg6vh9z23sx18m0000gn/T/ipykernel_83716/661470640.py:8: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

